<a href="https://colab.research.google.com/github/kalekarnn/CNN-Components-Back-Propagation/blob/main/assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
          nn.Conv2d(1, 32, 5, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(32),
          nn.Dropout(0.05),
          nn.Conv2d(32, 24, 3, stride=1, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(24),
          nn.Dropout(0.1)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(24, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 24, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(24),
            nn.Dropout(0.1)
        )

        self.conv4 = nn.Sequential(
              nn.Conv2d(24,16,3,stride=1,padding=1),
              nn.ReLU(),
              nn.BatchNorm2d(16)
          )


        self.conv5 = nn.Sequential(
            nn.Conv2d(16, 10, 3, stride=1)
        )

        self.gap = nn.AvgPool2d(5)



    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        x = self.gap(x)
        x = x.view(-1, 10)
        x = F.log_softmax(x, dim=1)
        return x

In [3]:
from torchsummary import summary

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             832
              ReLU-2           [-1, 32, 26, 26]               0
       BatchNorm2d-3           [-1, 32, 26, 26]              64
           Dropout-4           [-1, 32, 26, 26]               0
            Conv2d-5           [-1, 24, 26, 26]           6,936
              ReLU-6           [-1, 24, 26, 26]               0
       BatchNorm2d-7           [-1, 24, 26, 26]              48
           Dropout-8           [-1, 24, 26, 26]               0
            Conv2d-9           [-1, 16, 13, 13]           3,472
             ReLU-10           [-1, 16, 13, 13]               0
      BatchNorm2d-11           [-1, 16, 13, 13]              32
          Dropout-12           [-1, 16, 13, 13]               0
           Conv2d-13             [-1, 24, 7, 7]           3,480
             ReLU-14             [-1, 2

In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 191MB/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 45.4MB/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



...

100%|██████████| 1.65M/1.65M [00:00<00:00, 36.8MB/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.56MB/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [11]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.11945220082998276 batch_id=468: 100%|██████████| 469/469 [02:13<00:00,  3.51it/s]



Test set: Average loss: 0.0767, Accuracy: 9784/10000 (97.84%)



loss=0.17083798348903656 batch_id=468: 100%|██████████| 469/469 [02:16<00:00,  3.45it/s]



Test set: Average loss: 0.0476, Accuracy: 9857/10000 (98.57%)



loss=0.060158368200063705 batch_id=468: 100%|██████████| 469/469 [02:12<00:00,  3.53it/s]


...


Test set: Average loss: 0.0180, Accuracy: 9948/10000 (99.48%)



loss=0.0019424442434683442 batch_id=468: 100%|██████████| 469/469 [02:12<00:00,  3.53it/s]



Test set: Average loss: 0.0177, Accuracy: 9947/10000 (99.47%)



loss=0.044174712151288986 batch_id=468: 100%|██████████| 469/469 [02:12<00:00,  3.55it/s]



Test set: Average loss: 0.0172, Accuracy: 9942/10000 (99.42%)

